In [62]:
# importing libraries

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import random
import time
from bs4 import BeautifulSoup
import bs4
from selenium.common.exceptions import TimeoutException
import pandas as pd
import os

In [64]:
url = 'https://www.glassdoor.co.in/Job/india-data-analyst-jobs-SRCH_IL.0,5_IN115_KO6,18.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=Data%2520&typedLocation=India&context=Jobs&dropdown=0'
driver = webdriver.Chrome()
driver.get(url)

# create empty hrml list

html_list = []

# initialize the number of elements loaded to be zero

num_elements_loaded = 0

# intialising the loop

while True:
    timeout = random.randint(2, 5) # seting a random value for the timeout for every iteration

    # The num_elements variable is used to count the number of elements that have been loaded.

    num_elements = len(driver.find_elements(by=By.CSS_SELECTOR, value=".css-1vg6q84.e1tk4kwz4"))
    num_elements_loaded += num_elements # add the len of num_elemment to number of elements loaded in every iteraion
    print(f'number of elements found: {num_elements_loaded}')

    # If 500 elements are loaded, break the loop

    if num_elements_loaded >= 200:
        print(f'Elements loaded {num_elements_loaded}, breaking the loop.')
        break

    # # Wait for a random amount of time before reloading more elements.

    time.sleep(timeout) 
    
    # Scroll down to load more elements

    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait for more elements to load

    try:
        WebDriverWait(driver, 1000).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".css-1vg6q84.e1tk4kwz4")))
    except TimeoutException:
        # If timeout occurs, increase the timeout and continue scrolling

        continue

    # Store the HTML of the current scroll in a list

    html_list.append(driver.page_source)

# quit the driver

driver.quit()

number of elements found: 1
number of elements found: 2
number of elements found: 3
number of elements found: 4
number of elements found: 5
number of elements found: 6
number of elements found: 7
number of elements found: 8
number of elements found: 9
number of elements found: 10
number of elements found: 11
number of elements found: 12
number of elements found: 13
number of elements found: 14
number of elements found: 15
number of elements found: 16
number of elements found: 17
number of elements found: 18
number of elements found: 19
number of elements found: 20
number of elements found: 21
number of elements found: 22
number of elements found: 23
number of elements found: 24
number of elements found: 25
number of elements found: 26
number of elements found: 27
number of elements found: 28
number of elements found: 29
number of elements found: 30
number of elements found: 31
number of elements found: 32
number of elements found: 33
number of elements found: 34
number of elements foun

In [65]:
for i, html in enumerate(html_list):
    # Parse the HTML string using BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')
    # Write the prettified HTML to a file with a unique filename
    with open(f'glassdoor_output{i}.html', 'w', encoding='utf-8') as file:
        file.write(soup.prettify())

In [66]:
# Define the extraction functions

def text_extractor(node):
    return node.text
def string_extractor(node):
    if type(node.contents[0]) is bs4.element.NavigableString:
        return node.contents[0]
    else:
        return None

In [67]:
# create classes dictionary

classes = {
    'rating': {
        'element': 'span',
        'cls': 'css-2lqh28 e1cjmv6j1',
        'extr': string_extractor
    },
    'company_name': {
        'element': 'a',
        'cls': 'css-l2wjgv e1n63ojh0 jobLink',
        'extr': text_extractor
    },
    'job_position': {
        'element': 'a',
        'cls': 'jobLink css-1rd3saf eigr9kq2',
        'extr': text_extractor
    },
    'location': {
        'element': 'span',
        'cls': 'css-3g3psg pr-xxsm css-iii9i8 e1rrn5ka0',
        'extr': text_extractor
    },
    'salary_estimate': {
        'element': 'span',
        'cls': 'css-1xe2xww e1wijj242',
        'extr': text_extractor
    }
}

In [68]:
# Create an empty list to store the extracted data
data = []

# Loop through the html files and extract data from each file
for i in range(199):
    filename = f'glassdoor_output{i}.html'
    with open(filename, 'r', encoding='utf-8') as file:
        html = file.read()
    soup = BeautifulSoup(html, 'html.parser')
    result = {}

    # Loop through the classes dictionary and extract data for each class
    for key in classes:
        collection = soup.find_all(classes[key]['element'], class_=classes[key]['cls'])
        if collection:
            result[key] = [classes[key]['extr'](c) for c in collection]
        else:
            result[key] = [None] * len(data) # Fill in missing values with None

    # Ensure that the length of the extracted data is consistent for all classes
    max_len = max([len(result[key]) for key in result])
    for key in result:
        if len(result[key]) < max_len:
            result[key] += [None] * (max_len - len(result[key]))

    # result['file'] = filename # Add a new column to indicate which file the data is from
    data.append(result) # Append the result dictionary to the data list

# Concatenate the data from all files into a single DataFrame
df = pd.concat([pd.DataFrame(d) for d in data], ignore_index=True)


In [69]:
# replace \n from the dataframe

df = df.replace('\n', '', regex=True)
df.reset_index()

,index,rating,company_name,job_position,location,salary_estimate
0,0,4.1,S&P Global,Data Analyst,India,₹3L - ₹6L ...
1,1,4.2,Airtel India,Data Analyst,Gurgaon,₹5L - ₹9L ...
2,2,4.0,Barclays,Analyst - RFT Data Operations ...,New Delhi,₹8L - ₹8L ...
3,3,4.2,Airtel India,Business Analyst - Messaging ...,Gurgaon,₹40T - ₹45T ...
4,4,3.2,Glamyo Health,Data Analyst,Connaught Place,₹7L - ₹11L ...
...,...,...,...,...,...,...
5965,5965,None,Airtel India,Analyst,Gurgaon,None
5966,5966,None,Indigifts Pvt. Ltd. ...,Data Analyst,Jaipur,None
5967,5967,None,BNY Mellon,"Analyst, Data Management & Qua...",Maharashtra,None
5968,5968,None,SSPL,Data Analyst,New Delhi,None


In [71]:
# Write the DataFrame to a CSV file
df.to_csv('glassdoor_data_analyst_jobs_backup.csv', index=False)

In [72]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [73]:
def get_jobs(keyword, num_jobs, verbose):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path="C:/Users/9ayus/Documents/da_salary_project/chromedriver", options=options)
    driver.set_window_size(1120, 1000)

    url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword="' + keyword + '"&locT=C&locId=1147401&locKeyword=San%20Francisco,%20CA&jobType=all&fromAge=-1&minSalary=0&includeNoSalaryJobs=true&radius=100&cityId=-1&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0'
    driver.get(url)
    jobs = []

    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(10)

        #Test for the "Sign Up" prompt and get rid of it.
        try:
            driver.find_element(By.CLASS_NAME, "selected").click()
        except ElementClickInterceptedException:
            pass

        time.sleep(.1)

        try:
            driver.find_element(By.CLASS_NAME, "ModalStyle__xBtn___29PT9").click()  #clicking to the X.
        except NoSuchElementException:
            pass

        
        #Going through each job in this page
        job_buttons = driver.find_elements(By.CLASS_NAME, "jl")  #jl for Job Listing. These are the buttons we're going to click.
        for job_button in job_buttons:  

            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break

            job_button.click()  #You might 
            time.sleep(5)
            collected_successfully = False
            
            while not collected_successfully:
                try:
                    company_name = driver.find_element(By.XPATH, './/div[@class="employerName"]').text
                    location = driver.find_element(By.XPATH, './/div[@class="location"]').text
                    job_title = driver.find_element(By.XPATH, './/div[contains(@class, "title")]').text
                    job_description = driver.find_element(By.XPATH, './/div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                except:
                    time.sleep(10)

            try:
                salary_estimate = driver.find_element(By.XPATH, './/span[@class="gray small salary"]').text
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."
            
            try:
                rating = driver.find_element(By.XPATH, './/span[@class="rating"]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
                driver.find_element(By.XPATH, './/div[@class="tab" and @data-tab-type="overview"]').click()

                try:
                    #<div class="infoEntity">
                    #    <label>Headquarters</label>
                    #    <span class="value">San Francisco, CA</span>
                    #</div>
                    headquarters = driver.find_element(By.XPATH, './/div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
                except NoSuchElementException:
                    headquarters = -1

                try:
                    size = driver.find_element(By.XPATH, './/div[@class="infoEntity"]//label[text()="Size"]//following-sibling::*').text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element(By.XPATH, './/div[@class="infoEntity"]//label[text()="Founded"]//following-sibling::*').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element(By.XPATH, './/div[@class="infoEntity"]//label[text()="Type"]//following-sibling::*').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element(By.XPATH, './/div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element(By.XPATH, './/div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element(By.XPATH, './/div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*').text
                except NoSuchElementException:
                    revenue = -1

                try:
                    competitors = driver.find_element(By.XPATH, './/div[@class="infoEntity"]//label[text()="Competitors"]//following-sibling::*').text
                except NoSuchElementException:
                    competitors = -1

            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                headquarters = -1
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
                competitors = -1

                
            if verbose:
                print("Headquarters: {}".format(headquarters))
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("Competitors: {}".format(competitors))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Headquarters" : headquarters,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue,
            "Competitors" : competitors})
            #add job to jobs

        #Clicking on the "next page" button
        try:
            driver.find_element(By.XPATH, './/li[@class="next"]//a').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.

In [74]:
#This line will open a new chrome window and start the scraping.
df = get_jobs("data scientist", 5, False)
df

C:\Users\9ayus\AppData\Local\Temp\ipykernel_17732\3882706317.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="C:/Users/9ayus/Documents/da_salary_project/chromedriver", options=options)


Scraping terminated before reaching target number of jobs. Needed 5, got 0.


""


In [76]:
from bs4 import BeautifulSoup
import requests

# Send a request to the webpage
url = 'https://www.glassdoor.co.in/Job/india-data-analyst-jobs-SRCH_IL.0,5_IN115_KO6,18.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=Data%2520&typedLocation=India&context=Jobs&dropdown=0'
response = requests.get(url)

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Find all the clickable elements on the page
clickable_elements = soup.find_all(['a', 'button'], href=True, onclick=True)
for element in clickable_elements:
    print(element)


In [77]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import ElementClickInterceptedException, NoSuchElementException
import time
import pandas as pd

def get_clickable_elements(url):
    '''Returns all clickable elements on a webpage using BeautifulSoup'''
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    clickable_elements = soup.find_all(['a', 'button'], href=True, onclick=True)
    return clickable_elements

def get_jobs(keyword, num_jobs, verbose):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    # Get all clickable elements on the webpage
    url = 'https://www.glassdoor.com/Job/jobs.htm?'
    url_params = {
        'sc.keyword': keyword,
        'locT': 'C',
        'locId': '1147401',
        'locKeyword': 'San Francisco, CA',
        'jobType': 'all',
        'fromAge': '-1',
        'minSalary': '0',
        'includeNoSalaryJobs': 'true',
        'radius': '100',
        'cityId': '-1',
        'minRating': '0.0',
        'industryId': '-1',
        'sgocId': '-1',
        'seniorityType': 'all',
        'companyId': '-1',
        'employerSizes': '0',
        'applicationType': '0',
        'remoteWorkType': '0'
    }
    url += '&'.join([f"{k}={v}" for k, v in url_params.items()])
    clickable_elements = get_clickable_elements(url)

    # Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    # Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    # options.add_argument('headless')
    
    # Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path="C:/Users/9ayus/Documents/da_salary_project/chromedriver", options=options)
    driver.set_window_size(1120, 1000)

    driver.get(url)
    jobs = []
    job_urls = []

    while len(job_urls) < num_jobs:  # If true, should still be looking for new jobs.

        # Let the page load. Change this number based on your internet speed.
        # Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(10)

        # Test for the "Sign Up" prompt and get rid of it.
        try:
            driver.find_element(By.CLASS_NAME, "selected").click()
        except ElementClickInterceptedException:
            pass

        time.sleep(.1)

        try:
            driver.find_element(By.CLASS_NAME, "ModalStyle__xBtn___29PT9").click()  # clicking to the X.
        except NoSuchElementException:
            pass

        # Going through each job in this page
        job_buttons = driver.find_elements(By.CLASS_NAME, "jl")  # jl for Job Listing. These are the buttons we're going to click.
        for job_button in job_buttons:

            if len(job_urls) >= num_jobs:
                break

            job_button.click()  # You might
            time.sleep(5)
            collected_successfully = False

            while not collected_successfully:
                try:
                    company_name = driver.find_element(By.XPATH, './/div[@class="employerName"]').text
                    location = driver.find_element(By.XPATH, './/div[@class="location"]').text
                    job_title = driver.find_element(By.XPATH, './/div[contains(@class, "title")]').text
                    job_description = driver.find_element(By.XPATH, './/div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                except:
                    time.sleep(10)

            try:
                salary_estimate = driver.find_element(By.XPATH, './/span[@class="gray small salary"]').text
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."
            
            try:
                rating = driver.find_element(By.XPATH, './/span[@class="rating"]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
                driver.find_element(By.XPATH, './/div[@class="tab" and @data-tab-type="overview"]').click()

                try:
                    #<div class="infoEntity">
                    #    <label>Headquarters</label>
                    #    <span class="value">San Francisco, CA</span>
                    #</div>
                    headquarters = driver.find_element(By.XPATH, './/div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
                except NoSuchElementException:
                    headquarters = -1

                try:
                    size = driver.find_element(By.XPATH, './/div[@class="infoEntity"]//label[text()="Size"]//following-sibling::*').text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element(By.XPATH, './/div[@class="infoEntity"]//label[text()="Founded"]//following-sibling::*').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element(By.XPATH, './/div[@class="infoEntity"]//label[text()="Type"]//following-sibling::*').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element(By.XPATH, './/div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element(By.XPATH, './/div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element(By.XPATH, './/div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*').text
                except NoSuchElementException:
                    revenue = -1

                try:
                    competitors = driver.find_element(By.XPATH, './/div[@class="infoEntity"]//label[text()="Competitors"]//following-sibling::*').text
                except NoSuchElementException:
                    competitors = -1

            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                headquarters = -1
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
                competitors = -1

                
            if verbose:
                print("Headquarters: {}".format(headquarters))
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("Competitors: {}".format(competitors))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Headquarters" : headquarters,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue,
            "Competitors" : competitors})
            #add job to jobs

        #Clicking on the "next page" button
        try:
            driver.find_element(By.XPATH, './/li[@class="next"]//a').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.


In [78]:
#This line will open a new chrome window and start the scraping.
df = get_jobs("data scientist", 5, False)
df

C:\Users\9ayus\AppData\Local\Temp\ipykernel_17732\558897803.py:52: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="C:/Users/9ayus/Documents/da_salary_project/chromedriver", options=options)


Scraping terminated before reaching target number of jobs. Needed 5, got 0.


""
